## Strategy Idea 2 : "Cointegration - pairs trading"

__Section 0: Setup__ Importing packages/reading in data etc.

__Section 1 : Idea__ 

- __1.1__ Strategy idea

- __1.2__ Origin of idea. Context/Reasoning for strategy to work e.g. use in financial markets?

__Section 2 : Exploration__

- __2.1__ Exploratory Data Analysis. e.g plots of price/volumes that could show strategy working, how much potential.

- __2.2__ Define some 'strategy metrics'. Metrics that can can you use to gauge if this strategy will work i.e no.price points above a certain threshold that is profitable. Metrics could show how often there is an opportunity to make a trade and how much 'value' is in an opportunity e.g. how much is there a price swing?


__Section 3 : Strategy testing__

- __3.1__ Testing strategy on previous data. 

- __3.2__ State any assumptions made by testing.

- __3.3__ Model refinements. How could strategy be optimised? Careful : is this backfitting/overfitting - what measures taken to negate this e.g. bootstrapping?

- __3.4__ Assessing strategy. P/L on data sample? ROI? variance in results? longest losing run?

__Section 4 : Practical requirements__

- __4.1__ Identify if this edge is ‘realisable’? What methods will you apply to extract this value? e.g. applying a hedge function


- __4.2__ Is it possible to quantify the potential profit from the strategy? Consideration : How long will it take to obtain this? How 'risky' is it? e.g. if something did go wrong, how much do we lose? 

- __4.3__ Strategy limitations. The factors that could prevent strategy working e.g. practical considerations e.g. reacting quick enough to market updates, volume behind a price, size of bankroll needed


__Section 5: Potential limitations__

- __5.1__ What is our 'competition' - if not quantifiable, do we suspect people are doing the same thing? 

- __5.2__ So what's our edge? Identify ways of finding this edge in future? e.g what features are there? Are they predictive? Is there a certain 'market/runner' profile?





Notes (to do)
* Use lay prices as well (currently only using back prices, but two lay bets are made per pairs trade

### Section 0 : Setup

In [1]:
# importing packages
from pathlib import Path, PurePath 

import pandas as pd
import numpy as np
import statsmodels.api as sm

import matplotlib.pyplot as plt
import seaborn as sns

import itertools

import utils

In [2]:
def payout(bp, bs, lp, ls, c = 0):
    if ls == '?':
        ls = lay_hedge_stake(bp, bs, lp, c)
    elif bs == '?':
        bs = bet_hedge_stake(lp, ls, bp, c)
    loss_side = - bs + ls * (1 - c) 
    win_side = (bp - 1) * bs * (1 - c) - (lp - 1) * ls
    return win_side, loss_side 

def lay_hedge_stake(bp, bs, lp, c):
    return (((bp - 1) * bs * (1 - c)) + bs) / (lp)

def bet_hedge_stake(lp, ls, bp, c):
    return ls * (lp - c) / (bp * (1 - c) + c)

In [3]:
# reading in data
project_dir = Path.cwd().parents[2]
data_dir = project_dir / 'data' / 'processed' / 'api' / 'advanced' / 'adv_data.csv'
df = pd.read_csv(data_dir, index_col = 0)
print(df.shape)

# defining variables
back_prices = [col for col in df.columns if 'BP' in col]
back_sizes = [col for col in df.columns if 'BS' in col]
lay_prices = [col for col in df.columns if 'LP' in col]
lay_sizes = [col for col in df.columns if 'LS' in col]

df.head(2)

(13073, 307)


,SelectionId,MarketId,Venue,Distance,RaceType,BSP,NoRunners,BS:T-60,BS:T-59,BS:T-58,...,LS:T+5,LS:T+6,LS:T+7,LS:T+8,LS:T+9,LS:T+10,LS:T+11,LS:T+12,LS:T+13,LS:T+14
0,11986132,1.169028,Huntingdon,20.0,Chase,8.33,9,16.43,24.51,26.57,...,10.08,11.15,5.44,7.09,14.16,19.53,3.12,3.31,0.68,0.68
1,16800725,1.169028,Huntingdon,20.0,Chase,3.68,9,15.43,25.74,57.82,...,29.87,221.22,43.23,43.10,13.53,26.15,13.60,74.30,419.52,23082.10


### Section 1 : Pairs trading

__1.1__ **- Idea**

Pairs trading is a statistical arbitrage strategy whereby the trader attempts to exploit the tendency of some pairs of assets to have similar price changes. For example, consider two telecoms companies of similar size and profitability who are exposed to the same market conditions. Investors/traders see this similarity, and as such their prices may react in the same way to news. If there is a quantifiable relationship where the prices have a 'spread' of X on average, if the shares deviate from this average such that one is undervalued and one is overvalued, a long-short strategy can profit from a return to the average.

[Bebbington, PA (2017)](https://discovery.ucl.ac.uk/id/eprint/1563501/) looks at pairs trading in horse racing markets. 

The following outlines Bebbington's method for analysing this strategy.

* The 'signals' are the best back or lay price available at a given timestamp.
* A z-score transformation of the log of the decimal odds is used to standardise prices. This makes the relative directional movement in different prices comparable by accounting for their respective variances. 
* Non-overlapping windows of data, for example, price observations 1-5, 6-10, 11-15, are analysed to find pairs of horses. 
* Pairs are discovered by analysing the sum of squared distances between two horses' prices throughout time. Those that move the least relative to each other are the best candidates for pairs.
* Once pairs are identified, the 'spread' between their prices (on average, or at the end of each window) is compared to a minimum size requirement for a bet to be made, $\phi$.
* The 'hedging ratio' is found using an OLS regression of the price of one of the horses on the price of the other. Since the two prices are pairs but will have different variances and absolute values, their movement relative to eachother must be considered to make the strategy 'cost neutral'. It is also used to define the stake size. 
* The final observed spread indicates which on which horse a 'back-to-lay' hedge must be made (that which is expected to be backed in) and on which a 'lay-to-back' hedge must be made (that which is expected to drift). This spread is compared to an interval [?], likely a confidence interval of past spreads or simply the interval of observed spreads. If the spread is greater than usual or smaller than usual, the bets are placed. 
* Trades occur at the end of each window of data with a 5 second delay. This is used to simulate a method where the algorithm is reacting to live data. In our analysis, we look at prices for the first 30 price points and make bets in the remaining 30 periods.
* In the paper it appears that both sides of the hedge bet are made at the same point in time.

These steps are attempted below.

__1.2__ **- Setup**

**Data**

The following example will be set up with a random race and will identify tradeable pairs (or that there are none). Three DataFrames are created: (1) the unchanged race sample DataFrame with one row per horse and data going along in columns, (2) a back prices DataFrame with one column per horse and prices going through time in rows, (3) the same for lay prices. This analysis looks at prices before the race begins.

There are 60 price data points for each horse, finishing at the begining of the race.

Variables:
* $BP_{t}^{i}$ is back price for horse i at time t.
* $LP_{t}^{i}$ is lay price for horse i at time t.

In [4]:
sample_df = df[df['MarketId'] == df['MarketId'].sample(1).item()]

bp_df = sample_df[['SelectionId'] + back_prices].copy()
new_cols = bp_df.columns.str.replace("[BP:T]", "").str.replace("[+]", "")
bp_df.rename(columns = dict(zip(bp_df.columns, new_cols)), inplace = True)
bp_t_df = bp_df.T.copy()
bp_t_df.columns = ["h" + str(int(column)) for column in bp_t_df.iloc[0]]
bp_t_df = bp_t_df.iloc[1:-15] # using the 60 pre-off price data points
bp_t_df.reset_index(drop=True, inplace=True)

lp_df = sample_df[['SelectionId'] + lay_prices].copy()
new_cols = lp_df.columns.str.replace("[LP:T]", "").str.replace("[+]", "")
lp_df.rename(columns = dict(zip(lp_df.columns, new_cols)), inplace = True)
lp_t_df = lp_df.T.copy()
lp_t_df.columns = ["h" + str(int(column)) for column in lp_t_df.iloc[0]] #rename columns to horse ids
lp_t_df = lp_t_df.iloc[1:-15] #remove horse ids, remove inplay data
lp_t_df.reset_index(drop=True, inplace=True)

# bsp_df = sample_df[['BSP']].copy()
# bsp_df['min_bp'] = sample_df['BSP'].apply(lambda x: round(utils.back_hedge_min_bp(x, 0.05), 2))
# bsp_df['max_lp'] = sample_df['BSP'].apply(lambda x: round(utils.lay_hedge_max_lp(x, 0.05), 2))    

bp_t_df.head(2)

,h10163711,h16117335,h20179313,h21104332,h22403664,h26641007,h27535763,h4003026
0,19.50,8.0,46.43,16.5,12.0,1.69,190.0,14.0
1,19.61,7.8,48.00,16.5,12.0,1.69,190.0,14.0


__1.3__ **- Z-score transformation**

Bebbington standardises prices by taking the natural logarithm of each price and then standardising it with a Z-score transformation. 

Taking $P_{t}^{i} = ln(BP_{t}^{i})$, this means finding 

#### $P_{t}^{'(i)} = \frac{P_{t}^{i}-\overline{P}_{t}^{i}}{\sigma^{(i)}}$

where $\sigma^{(i)}$ is the standard deviation of the horse's price throughout the time series.

The Z-score transformation gives the relationship between an individual data point in the sample relative to that of the population mean and standard deviation. This means that variations are comparable between horses.

The following will look at the first 30 observations.

In [5]:
z_bp_df = bp_t_df.copy()
z_bp_df = z_bp_df[:30] #first 30 observations
z_bp_df = np.log(z_bp_df)

for column in z_bp_df.columns:
    mean = z_bp_df[column].mean()
    sd = np.std(z_bp_df[column], ddof = 1)
    z_bp_df[column] = z_bp_df[column].apply(lambda x: (x - mean) / sd)
    
z_bp_df.head(2)

,h10163711,h16117335,h20179313,h21104332,h22403664,h26641007,h27535763,h4003026
0,1.451617,2.101047,-0.635630,1.153069,-2.051518,-0.280802,1.148649,1.528065
1,1.606916,1.363358,0.215995,1.153069,-2.051518,-0.280802,1.148649,1.528065


__1.4__ **- Sum of squared distances**

Following Bebbington, to select pairs, we create a matrix (DataFrame, in this case) of the sum of squared distances between pairs of horses throughout the time series.

$\Theta _{ij} = \left\{\begin{matrix}
\sum_{M}^{t=1}(P_{t}^{'(i)} - P_{t}^{'(j)})^{2}, &i\neq j\\ 
 0, i=j& 
\end{matrix}\right.$

In [6]:
ids = [column for column in z_bp_df.columns]

matrix = z_bp_df.iloc[0:0].copy()

matrix.insert(0, "horse", np.array(ids))
matrix = matrix.set_index("horse", drop = False)
del matrix["horse"]

for column in matrix.columns:
    for row in matrix.index:
        if column == row:
            matrix.loc[row, column] = np.nan
        else: 
            matrix.loc[row, column] = ((z_bp_df[row] - z_bp_df[column]) ** 2).sum() or np.nan
            
for x in range(len(ids)):
    for y in range(x, len(ids)):
        matrix.iloc[x, y] = np.nan
        
matrix

,h10163711,h16117335,h20179313,h21104332,h22403664,h26641007,h27535763,h4003026
horse,,,,,,,,
h10163711,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
h16117335,28.233027,NaN,NaN,NaN,NaN,NaN,NaN,NaN
h20179313,61.114044,32.663370,NaN,NaN,NaN,NaN,NaN,NaN
h21104332,46.560153,34.091779,68.108368,NaN,NaN,NaN,NaN,NaN
h22403664,103.124656,105.536149,63.222708,84.208128,NaN,NaN,NaN,NaN
h26641007,69.778177,101.531219,86.147787,70.059679,37.405348,NaN,NaN,NaN
h27535763,42.675603,55.851381,65.060994,60.074608,56.933814,63.700918,NaN,NaN
h4003026,48.117953,25.424362,68.149755,40.541261,86.148868,84.084856,53.016157,NaN


In [7]:
horse_x = matrix.min(axis=1).idxmin()
horse_y = matrix.min().idxmin()
sss = matrix.min().min()

print(f"Pair found: horse {horse_x} and horse {horse_y} with sum of squared spreads equal to {sss}.")

Pair found: horse h4003026 and horse h16117335 with sum of squared spreads equal to 25.42436206727571.


__1.5__ **- Regression of prices of horse Y on horse X**

With $x_{t} = \left \{P_{1}^{X} + P_{2}^{X} + , ... , P_{M}^{X} \right \}$ and  $y_{t} = \left \{P_{1}^{Y} + P_{2}^{Y} + , ... , P_{M}^{Y} \right \}$ where $M$ is the final time period in the window, we carry out the OLS regression of $y_{t}$ on $x_{t}$. The estimate of $\beta$ is the hedging ratio, giving the relative holding of $x_{t}$ for a cost-neutral hedge position.

$y_{t} = \beta x_{t} + \varepsilon_{t}$

Using this estimation and the final end of window observations at time $M$ we get the spread at the end of the window: $\varepsilon_{M} = y_{M} - \hat{\beta} x_{M}$. If $\varepsilon_{M}$ is outside of an interval of past spread values such that if the spread returns to the mean a hedge bet will be profitable, bets can be made.

In [8]:
#regression setup
reg_df = bp_t_df[[horse_y, horse_x]][:30].copy() #non-standardised prices
reg_df = np.log(reg_df) 
reg_df['const'] = 1

#regression fit and results
reg = sm.OLS(endog=reg_df[horse_y], exog=reg_df[['const', horse_x]], missing='drop')

results = reg.fit()

constant = results.params[0]
beta = results.params[1]

#estimated final period (T=30) spread
spread = reg_df[horse_y].iloc[29].item() - constant - beta * reg_df[horse_x].iloc[29].item()

print(f"Final period estimated spread (in log prices) epsilon = {spread}.")

#Positive spread: horse Y has drifted from the mean, horse X has been backed in. If mean reversion occurs horse Y will be backed in and horse X will drift. 
# -> Back-to-lay hedge Y and lay-to-back hedge X
#Negative spread: horse X has drifted from the mean, horse Y has been backed in. If mean reversion occurs horse X will be backed in and horse Y will drift. 
# -> Back-to-lay hedge X and lay-to-back hedge Y

Final period estimated spread (in log prices) epsilon = -0.07810240582111483.


__1.6.1__ **- Pairs trade**

Bebbing describes the following steps for the pairs trade:

If the final estimated residual $\varepsilon_{M}$ is:
* Less than the lower bound of the threshold: back horse Y with £1, lay horse X with £$\beta$.
* Greater than the upper bound of the threshold: lay horse Y with £1 and back horse X with £$\beta$.

Bebbington describes using a self-imposed delay of 5 seconds to emulate the delays in placing a bet on the Betfair Exchange. Since each time step in this dataset before the beginning of the race is 2 minutes, 1 step will be used. The prices at time 31 (index = 30) will be used.

Put alone, this strategy doesnt make sense. The bettor is still exposed to the result of the race without closing out each position with an opposing bet on each horse after some period of time.

__1.6.2__ **- Pairs trade 2**

In a pairs trade with two financial assets (say, company shares) the objective is to go long on the share whose price is expected to increase, while shorting the share whose price is expected to decrease. In both cases the positions would be closed out: the asset is sold or bought back (in the case of the short-sale) once there is a gain or possibly after a set interval. In the prior example, the closing out of each position should be done via an opposite back/lay.

If the final estimated residual $\varepsilon_{M}$ is:
* Less than the lower bound of the threshold: back horse Y with £1, lay horse X with £$\beta$. **After k periods, lay horse Y and back horse X with the optimal stakes given the prevailing prices, as defined in utils.py.**
* Greater than the upper bound of the threshold: lay horse Y with £1 and back horse X with £$\beta$. **After k periods, back horse Y and lay horse X with the optimal stakes.**

In [9]:
# k = 35

# if spread < 0:
#     #back to lay Y
#     bp_y = bp_t_df[horse_y].iloc[30]
#     lp_y = bp_t_df[horse_y].iloc[k]
    
#     win_side_y, loss_side_y = utils.payout(bp_y, 1, lp_y, '?')
#     print(f"Win side = {win_side_y}. Loss side = {loss_side_y}.")
    
#     #lay to back X
#     lp_x = bp_t_df[horse_x].iloc[30]
#     bp_x = bp_t_df[horse_x].iloc[k]
    
#     win_side_x, loss_side_x = utils.payout(bp_x, '?', lp_x, beta)
#     print(f"Win side = {win_side_x}. Loss side = {loss_side_x}.")
        
# else:
#     #lay to back Y
#     lp_y = bp_t_df[horse_y].iloc[30]
#     bp_y = bp_t_df[horse_y].iloc[k]
    
#     win_side_y, loss_side_y = utils.payout(bp_y, '?', lp_y, 1)
#     print(f"Win side = {win_side_y}. Loss side = {loss_side_y}.")
    
#     #back to lay X
#     bp_x = bp_t_df[horse_x].iloc[30]
#     lp_x = bp_t_df[horse_x].iloc[k]
    
#     win_side_x, loss_side_x = utils.payout(bp_x, beta, lp_x, '?')
#     print(f"Win side = {win_side_x}. Loss side = {loss_side_x}.")

I believe Bebbington's methodology has flaws. The following attempts to use a more robust method for pairs trading.

### Alternative approach to pairs trading

__2.0__ **- [Herlemont (2004)](http://docs.finance.free.fr/DOCS/Yats/cointegration-en%5B1%5D.pdf) paper**

Herlemont describes in detail the econometrics of pairs trading for financial market assets. The following partly follows his commentary with some additional clarifications and discussion relating to horse racing.

**2.1 - Testing for mean reversion**

The aim is to identify odds that move together and whose spread is mean reverting. For the purposes of horse racing pairs, mean reversion is essential. Our objective is to capture prices whose spread has (temporarily) deviated from its mean. If this can be found, bets can be made to take advantage of the possible reversion.

A stochastic process $y_{t}$ that is weakly stationary has the following properties for all $t$:

* $E[y_{t}] = \mu < \infty$
* $var(y_{t}) = \gamma_{0} < \infty$
* $cov(y_{t}, y_{t-j}) = \gamma_{j} < \infty, j = 1, 2, 3 ...$

(constant mean, constant variance, covariance between two observations depends only on the distance in time between them)

A weakly stationary $I(0)$ series:
* Fluctuates around its mean with a finite variance that does not depend upon time.
* Is mean-reverting: it has tendency to return to its mean.
* Has limited memory; the effect of a shock dies out. Autocorrelations die out (fairly) rapidly.

With two horse's odds, $A_{t}$ and $B_{t}$, we look at $y_{t} = \log \frac{A_{t}}{B_{t}} = \log A_{t} - \log B_{t}$. This is once again the spread between the prices of the two horses, defined slightly differently. We want to find a pair which has a weakly stationary spread. We are interested in the ($AR(1)$) process 

$y_{t} = c + \theta y_{t-1} + \varepsilon_{t}$,

or the log odds ratio over time. If this is weakly stationary, it would suggest a mean reverting process. 

The three previous conditions, and a stability condition that $|\theta|<1$ (that the process $y_{t}$ is not a random walk or that it follows an eratic positive-to-negative pattern) must hold.
______

A Dickey-Fuller stationarity test can be carried out on the log ratio of the prices to test whether a process is weakly stationary. If we carry out the regression:

$\Delta y_{t} = \mu + \omega y_{t-1} + \varepsilon_{t}$

where the null hypothesis that $\omega = 0$ is that the 'true' relationship is $\Delta y_{t} = \mu + \varepsilon_{t} \Leftrightarrow y_{t} = \mu + y_{t-1} + \varepsilon_{t}$, or a random walk with starting point $y_{0} = \mu$.

If we can reject the null hypothesis, the price ratio is weakly stationary and thereby mean-reverting.

A Dickey-Fuller test is required for each possible pair of horses in a race, or $\frac{n(n-1)}{2}$ regressions, where $n$ is the number of horses.

While we are interested in the stochastic process $y_{t}$, we do not need to carry out the regression of $y_{t} = c + \theta y_{t-1} + \varepsilon_{t}$ for the purpose of finding pairs. This relationship between a pair of odds itself is not important to quantify. We are only interested in the features of the process. 
____

*In the previous analysis, the test for whether two odds formed a pair was to find the pair with the smallest sum of absolute differences over time in the standardised prices. That method would allow maximum 1 pair to be found per race, and the validity of that pair would not be confirmed statisticallyather. Rather, the pair's feasibilty for a trade would be tested for afterwards based on profitability. I have more confidence in the approach in this section.*

**2.2 - Screening pairs**

Herlemont describes rules to ensure that market neutrality is more achievable in pairs trading. The idea is to pick stocks with very similar characteristics like same industry and similar market betas, with the intention of minimising asymmetric shocks to the price of one stock and not the other. For example in the case of two stocks, the share on which you are long is a business heavily dependent on oil, while the other share is not, a surge in oil prices which dampens profitability of your long share will likely see its price fall, ruining the pairs trade. In the case of shares, the simplest solution would be to pick shares in similar industries with similar market betas (or with similar idiosyncratic risks).

For horses, the external factors influencing prices (news about runners, changing weather conditions, etc.) will usually always have asymmetric effects. This may be avoidable through picking horses with similar fundamental characteristics. However, this is very complicated. My hope is that the pair finding mechanism picks horses where this is already the case, because the market reacts the same way to news for these horse pairs.

We cannot follow a beta-based approach because there are not 'market-wide fluctuations' of the same sort. However, there is the fact that the implied probability of all horses in the market book is equal to approximately 1. Therefore, you could say that for a given change in implied probability for one horse, the sum of the changes in the odds of all the remaining horses is the negative the change for the given horse:

$\Delta O_{i} = - \sum_{j = 1, j \neq i}^{N_{h}} \Delta O_{j} $

There is therefore interdependence between all prices across the market. It's possible that this will cause an endogeneity problem in regressions between separate horses, as the changes in the dependent variable necessarily impact the explanatory variable. However, the impact is likely to be very small, and will be smaller the greater the number of horses. 

*In Bebbington's analysis, he describes that betting £1 on one of the horses and £$\beta$ on the other creates a market neutral bet. This is incorrect, and it appears that he has misunderstood hedging in this context. In that analysis, $\beta = \frac{y_{t}}{x_{t}}$, and therefore he is simply considering the ratio of the prices of the horses, the same ratio considered when determining the optimal stake for two given prices in a hedge. It is correct that on a single horse this creates a market neutral bet, however neutrality in horse racing means neutral to the outcome of the race. Any bet neutral to the race outcome is definitively neutral to the market. When betting on separate horses, the bets on each horse must be made neutral separately. Additionally, the use of $\beta$ in staking is unneccesary. Consider the case where £$BS$ has been bet on horse A at price $BP$. Now, horse A is priced at $LP$. The optimal stake to bet on LP is £$LS = \frac{BS * BP}{LP}$. In the aforementioned regression, $BS = 1$, hence $\beta = \frac{y_{t} * 1}{x_{t}}$ is the optimal stake only for bets of £1, otherwise it would be $S*\beta$. More importantly, using the estimated $\beta$ to find the an approximation of the optimal stake makes no sense when you can simply find the optimal stake with the aforementioned equation.*

**2.3 - Trading rules**

Timing rules must be added. 

Herlemont's basic rule is "to open a position when the ratio of two share prices hits the 2 rolling standard deviation [difference from the 130-day rolling mean] and close it when the ratio returns to the mean."

To avoid opening a position on stocks that are deviating from the mean and are going to deviate further, Herlemont describes that "the position is not opened when the ratio breaks the two-standard-deviations limit for the first time, but rather when it crosses it to revert to the mean again."

This can be achieved with the horse odds, of course in far smaller time scales. The current dataset is in 5-minute intervals for the three hours before a race; this should likely be expanded.

Stop losses should be included and trade length should also be limited.

Rules:
1. Trade on pairs whose spread is reapproaching the mean from a deviated position
2. Stop loss at x% of the initial position
3. Don't hold open pairs trades for longer than x hours. 

It should be possible to quantify the average length of time required for a mean reversion and therefore the maximum logical time to hold open a position by looking at past data.

**2.4 - Other tests and considerations**

1. It should be ensured that the regression results of one price on another are not spurious (as with the regression in 2.5). $\beta$ could be statistically meaningless if it is, meaning that it makes no sense to use it.
2. I will also test whether $y_{t} = c + \theta y_{t-1} + \varepsilon_{t}$ is $I(1)$, or difference stationary. If we can rule this out, this gives more confidence in the 'weak-stationarity' of the spread over time.
3. I will look out for $\omega$ in the DF test that are close to 1 yet pass the DF test. They will have lots of features of a random walk, so the pairs exercise might be meaningless.
4. Structural breaks (in this case, large instantaneous jumps in the spread) may make series that are stationary on either side of the break appear non-stationary. This is hard to account for in testing. 

In [57]:
#new sample
sample_df = df[df['MarketId'] == df['MarketId'].sample(1).item()]
sample_df.drop_duplicates(inplace=True)

bp_df = sample_df[['SelectionId'] + back_prices].copy()
new_cols = bp_df.columns.str.replace("[BP:T]", "").str.replace("[+]", "")
bp_df.rename(columns = dict(zip(bp_df.columns, new_cols)), inplace = True)
bp_t_df = bp_df.T.copy()
bp_t_df.columns = ["h" + str(int(column)) for column in bp_t_df.iloc[0]]
bp_t_df = bp_t_df.iloc[1:-15] # using the 60 pre-off price data points
bp_t_df.reset_index(drop=True, inplace=True)

lp_df = sample_df[['SelectionId'] + lay_prices].copy()
new_cols = lp_df.columns.str.replace("[LP:T]", "").str.replace("[+]", "")
lp_df.rename(columns = dict(zip(lp_df.columns, new_cols)), inplace = True)
lp_t_df = lp_df.T.copy()
lp_t_df.columns = ["h" + str(int(column)) for column in lp_t_df.iloc[0]] #rename columns to horse ids
lp_t_df = lp_t_df.iloc[1:-15] #remove horse ids, remove inplay data
lp_t_df.reset_index(drop=True, inplace=True)

C:\Users\Asus\anaconda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [58]:
#3.1 - testing for mean reversion

#using non-standardised log price data
log_bp = np.log(bp_t_df[:30]).copy()
log_bp.head()

,h13478734,h17247906,h19148736,h19252818,h19641188,h22121327,h24872,h26698821,h8319283,h8536472,h9659886
0,2.449279,3.091042,2.136531,1.481605,1.593309,1.824549,3.216874,4.094345,3.248823,4.084294,2.703373
1,2.484907,3.091042,2.128232,1.481605,1.609438,1.822935,3.218876,4.096176,3.242202,4.094345,2.674149
2,2.484907,3.144583,2.124654,1.460938,1.593309,1.824549,3.218876,4.077198,3.249599,4.174387,2.674149
3,2.484907,3.181797,2.112635,1.479329,1.570697,1.819699,3.213260,4.007333,3.287655,4.181897,2.674149
4,2.484907,3.258097,2.104134,1.474763,1.581038,1.803359,3.203965,4.007333,3.266522,4.248495,2.667228


In [59]:
#create dataframe where each column is log(horse a's prices) - log(horse b's prices). one new column for all n(n-1)/2 possible pairs

#use itertools to find all possible comination pairs
combos = list(itertools.combinations(log_bp.columns, 2))

#creating dataframe
for pair in combos:
    if pair == combos[0]:
        new_series = log_bp[pair[0]] - log_bp[pair[1]]
        dickey_fuller_df = pd.DataFrame(new_series)
    else:
        new_series = log_bp[pair[0]] - log_bp[pair[1]]
        dickey_fuller_df = pd.concat([dickey_fuller_df, new_series], axis=1)
        
#naming columns
dickey_fuller_df.columns = [pair[0] + "_" + pair[1] for pair in combos]

dickey_fuller_df['const'] = 1

dickey_fuller_df.head()

,h13478734_h17247906,h13478734_h19148736,h13478734_h19252818,h13478734_h19641188,h13478734_h22121327,h13478734_h24872,h13478734_h26698821,h13478734_h8319283,h13478734_h8536472,h13478734_h9659886,...,h24872_h8319283,h24872_h8536472,h24872_h9659886,h26698821_h8319283,h26698821_h8536472,h26698821_h9659886,h8319283_h8536472,h8319283_h9659886,h8536472_h9659886,const
0,-0.641763,0.312749,0.967675,0.855971,0.624730,-0.767594,-1.645065,-0.799543,-1.635015,-0.254093,...,-0.031949,-0.867420,0.513501,0.845522,0.010050,1.390972,-0.835471,0.545450,1.380922,1
1,-0.606136,0.356675,1.003302,0.875469,0.661972,-0.733969,-1.611270,-0.757295,-1.609438,-0.189242,...,-0.023326,-0.875469,0.544727,0.853975,0.001832,1.422028,-0.852143,0.568053,1.420196,1
2,-0.659677,0.360253,1.023969,0.891598,0.660357,-0.733969,-1.592292,-0.764692,-1.689481,-0.189242,...,-0.030723,-0.955511,0.544727,0.827599,-0.097189,1.403050,-0.924788,0.575450,1.500239,1
3,-0.696890,0.372272,1.005577,0.914210,0.665208,-0.728353,-1.522427,-0.802749,-1.696991,-0.189242,...,-0.074395,-0.968637,0.539111,0.719678,-0.174564,1.333185,-0.894242,0.613507,1.507749,1
4,-0.773190,0.380772,1.010144,0.903868,0.681548,-0.719059,-1.522427,-0.781616,-1.763589,-0.182322,...,-0.062557,-1.044530,0.536737,0.740811,-0.241162,1.340105,-0.981973,0.599294,1.581267,1


In [60]:
#dickey fuller test on each column

#regression fit and results in vertical dataframe format (column for pair id, column for dickey fuller test result)
dickey_fuller_results = {'pair' : [], 'coef' : [], 'critical_value' : []}

for column in dickey_fuller_df:
    if column == 'const':
        break
    reg = sm.OLS(endog = dickey_fuller_df[column].diff(), exog = dickey_fuller_df[['const', column]].shift(1), missing = 'drop')
    results = reg.fit()
    dickey_fuller_results['pair'].append(column)
    dickey_fuller_results['coef'].append(results.params[1])
    dickey_fuller_results['critical_value'].append(results.tvalues[1])

dickey_fuller_results_df = pd.DataFrame(dickey_fuller_results)

dickey_fuller_results_df

#compare to -3.58 from the MacKinnon tables for 1% significance level, -2.93 for 5% significance level

,pair,coef,critical_value
0,h13478734_h17247906,-0.142400,-1.812171
1,h13478734_h19148736,-0.100319,-2.388007
2,h13478734_h19252818,-0.087050,-2.050789
3,h13478734_h19641188,-0.129434,-1.496938
4,h13478734_h22121327,0.008812,0.174070
5,h13478734_h24872,0.005747,0.072271
6,h13478734_h26698821,-0.220010,-3.662027
7,h13478734_h8319283,-0.170112,-1.723562
8,h13478734_h8536472,-0.228423,-1.649025
9,h13478734_h9659886,0.008148,0.174761


In [61]:
#any pairs with a critical value less than have the null hypothesis rejected at the 1% significance level
if dickey_fuller_results_df['critical_value'].min() < - 3.58:
    pairs_df = dickey_fuller_results_df.loc[dickey_fuller_results_df['critical_value'] < - 3.58].copy() #all possible pairs
    print(f"Pairs found")
    pair_index = pairs_df['critical_value'].idxmin() #the most stationary pair 
    pair_cv = pairs_df['critical_value'].min()
    pair_ids = pairs_df['pair'].loc[pair_index]
    pair_coef = pairs_df['coef'].loc[pair_index]

    horse_a = pair_ids.split("_", 1)[0]
    horse_b = pair_ids.split("_", 1)[1]
    pair_df = bp_t_df[[horse_a, horse_b]]
    pair_df = pd.concat([pair_df, lp_t_df[[horse_a, horse_b]]], axis=1)
    pair_df.columns = [horse_a + "_bp", horse_b + "_bp", horse_a + "_lp", horse_b + "_lp"]
    
    #the following are all defined only in terms of BP
    pair_df['spread'] = pair_df[horse_a + "_bp"] - pair_df[horse_b + "_bp"]

    pair_spread_sd = np.std(pair_df['spread'][0:29], ddof = 1)
    pair_spread_mean = pair_df['spread'][0:29].mean()

    pair_df['deviation_2sd'] = np.where(abs(pair_df['spread']) - abs(pair_spread_mean) > 2 * pair_spread_sd, True, False)
    pair_df['deviation_1sd'] = np.where(abs(pair_df['spread']) - abs(pair_spread_mean) > pair_spread_sd, True, False)

    print("Pair row index: " + str(pair_index), ", Pair ids: " + str(pair_ids), ", Pair DF test critical value: " + str(pair_cv), ", Pair theta: " + str(pair_coef))
    print("Pair average spread: " + str(pair_spread_mean), ", Pair spread standard deviation: " + str(pair_spread_sd) + "\n")
    
else: print("No pairs.")

Pairs found
Pair row index: 6 , Pair ids: h13478734_h26698821 , Pair DF test critical value: -3.662027128375918 , Pair theta: -0.22000972730609739
Pair average spread: -43.04310344827586 , Pair spread standard deviation: 1.9916577865979523



In [62]:
pair_df

,h13478734_bp,h26698821_bp,h13478734_lp,h26698821_lp,spread,deviation_2sd,deviation_1sd
0,11.58,60.00,12.32,68.07,-48.42,True,True
1,12.00,60.11,12.50,70.00,-48.11,True,True
2,12.00,58.98,12.56,68.75,-46.98,False,True
3,12.00,55.00,12.77,62.95,-43.00,False,False
4,12.00,55.00,12.61,64.55,-43.00,False,False
5,12.39,55.00,12.93,65.00,-42.61,False,False
6,12.50,55.00,13.00,65.00,-42.50,False,False
7,12.50,55.00,13.00,65.00,-42.50,False,False
8,12.50,55.00,13.00,64.09,-42.50,False,False
9,12.50,55.00,13.00,60.00,-42.50,False,False


From looking at repeated versions of the table above, on occasions where the price has drifted more than two standard deviations from the mean, it has returned to the mean after around 5 time periods (10 minutes). I will look at a pairs trading strategy where the pairs trade is made upon deviation and closed 10 minutes after the deviation.

In [63]:
k = 5 #number of periods trade is kept open
 
open_trade_df = pair_df[pair_df['deviation_2sd'] == True].loc[30:] #so that we only consider data after the first 30 periods
open_trade_idx = open_trade_df.index[0] 
    
close_trade_idx = open_trade_idx + k

#to work out what the direction of the trade should be
#the spreads are always defined as horse_a - horse_b, so if the spread > average, btl horse_a and ltb horse_b. if below average, vice versa.
if (pair_df['spread'].iloc[open_trade_idx] > pair_spread_mean and pair_spread_mean > 0) or (pair_df['spread'].iloc[open_trade_idx] < pair_spread_mean and pair_spread_mean < 0):
    #back to lay A
    bp_a = pair_df[horse_a + "_bp"].iloc[open_trade_idx]
    lp_a = pair_df[horse_a + "_lp"].iloc[close_trade_idx]
    
    win_side_a, loss_side_a = payout(bp_a, 1, lp_a, '?')
    print(f"Win side = {win_side_a}. Loss side = {loss_side_a}.")
    
    #lay to back X
    lp_b = pair_df[horse_b + "_lp"].iloc[open_trade_idx]
    bp_b = pair_df[horse_b + "_bp"].iloc[close_trade_idx]
    
    win_side_b, loss_side_b = payout(bp_b, '?', lp_b, 1)
    print(f"Win side = {win_side_b}. Loss side = {loss_side_b}.")
    
else: 
    #lay to back Y
    lp_a = pair_df[horse_a + "_lp"].iloc[open_trade_idx]
    bp_a = pair_df[horse_a + "_bp"].iloc[close_trade_idx]
    
    win_side_a, loss_side_a = payout(bp_a, '?', lp_a, 1)
    print(f"Win side = {win_side_a}. Loss side = {loss_side_a}.")
    
    #back to lay X
    bp_b = pair_df[horse_b + "_bp"].iloc[open_trade_idx]
    lp_b = pair_df[horse_b + "_lp"].iloc[close_trade_idx]
    
    win_side_b, loss_side_b = payout(bp_b, 1, lp_b, '?')
    print(f"Win side = {win_side_b}. Loss side = {loss_side_b}.")

Win side = -0.0360962566844929. Loss side = -0.03609625668449201.
Win side = 0.12379521842534302. Loss side = 0.12379521842533481.


**Monte Carlo simulation**

n repetitions of the above with profit summed over all trades.

In [66]:
#setup
n = 1000 #number of iterations
k = 5 #number of 2 mintue periods trade is kept open (i.e. time expected for mean reversion to occur)

#results variables
profit = 0
num_pairs = 0
num_trades = 0

for i in range(n):
            
    if (i + 1) % 100 == 0:
        print(f"{i+1} of {n} iterations completed.")
    
    #new sample
    sample_df = df[df['MarketId'] == df['MarketId'].sample(1).item()]
    sample_df.drop_duplicates(inplace=True)

    bp_df = sample_df[['SelectionId'] + back_prices].copy()
    new_cols = bp_df.columns.str.replace("[BP:T]", "").str.replace("[+]", "")
    bp_df.rename(columns = dict(zip(bp_df.columns, new_cols)), inplace = True)
    bp_t_df = bp_df.T.copy()
    bp_t_df.columns = ["h" + str(int(column)) for column in bp_t_df.iloc[0]]
    bp_t_df = bp_t_df.iloc[1:-15] # using the 60 pre-off price data points
    bp_t_df.reset_index(drop=True, inplace=True)

    lp_df = sample_df[['SelectionId'] + lay_prices].copy()
    new_cols = lp_df.columns.str.replace("[LP:T]", "").str.replace("[+]", "")
    lp_df.rename(columns = dict(zip(lp_df.columns, new_cols)), inplace = True)
    lp_t_df = lp_df.T.copy()
    lp_t_df.columns = ["h" + str(int(column)) for column in lp_t_df.iloc[0]] #rename columns to horse ids
    lp_t_df = lp_t_df.iloc[1:-15] #remove horse ids, remove inplay data
    lp_t_df.reset_index(drop=True, inplace=True)

    #using non-standardised log price data
    log_bp = np.log(bp_t_df[:30]).copy()
    log_bp.head()

    #create dataframe where each column is log(horse a's prices) - log(horse b's prices). one new column for all n(n-1)/2 possible pairs
    #use itertools to find all possible comination pairs
    combos = list(itertools.combinations(log_bp.columns, 2))

    for pair in combos:
        if pair == combos[0]:
            new_series = log_bp[pair[0]] - log_bp[pair[1]]
            dickey_fuller_df = pd.DataFrame(new_series)
        else:
            new_series = log_bp[pair[0]] - log_bp[pair[1]]
            dickey_fuller_df = pd.concat([dickey_fuller_df, new_series], axis=1)

    #naming columns
    dickey_fuller_df.columns = [pair[0] + "_" + pair[1] for pair in combos]
    dickey_fuller_df['const'] = 1

    #dickey fuller test on each column
    dickey_fuller_results = {'pair' : [], 'coef' : [], 'critical_value' : []}

    for column in dickey_fuller_df:
        if column == 'const':
            continue
        reg = sm.OLS(endog = dickey_fuller_df[column].diff(), exog = dickey_fuller_df[['const', column]].shift(1), missing = 'drop')
        results = reg.fit()
        dickey_fuller_results['pair'].append(column)
        dickey_fuller_results['coef'].append(results.params[1])
        dickey_fuller_results['critical_value'].append(results.tvalues[1])

    dickey_fuller_results_df = pd.DataFrame(dickey_fuller_results)

    #any pairs with a critical value less than have the null hypothesis rejected at the 1% significance level
    if dickey_fuller_results_df['critical_value'].min() < - 3.58:
        pairs_df = dickey_fuller_results_df.loc[dickey_fuller_results_df['critical_value'] < - 3.58].copy()
        
        num_pairs += 1

        pair_index = pairs_df['critical_value'].idxmin()
        pair_cv = pairs_df['critical_value'].min()
        pair_ids = pairs_df['pair'].loc[pair_index]
        pair_coef = pairs_df['coef'].loc[pair_index]

        horse_a = pair_ids.split("_", 1)[0]
        horse_b = pair_ids.split("_", 1)[1]
        pair_df = bp_t_df[[horse_a, horse_b]]
        pair_df = pd.concat([pair_df, lp_t_df[[horse_a, horse_b]]], axis=1)
        pair_df.columns = [horse_a + "_bp", horse_b + "_bp", horse_a + "_lp", horse_b + "_lp"]

        #the following are all defined only in terms of BP
        pair_df['spread'] = pair_df[horse_a + "_bp"] - pair_df[horse_b + "_bp"]

        pair_spread_sd = np.std(pair_df['spread'][0:29], ddof = 1)
        pair_spread_mean = pair_df['spread'][0:29].mean()

        pair_df['deviation_2sd'] = np.where(abs(pair_df['spread']) - abs(pair_spread_mean) > 2 * pair_spread_sd, True, False)
        pair_df['deviation_1sd'] = np.where(abs(pair_df['spread']) - abs(pair_spread_mean) > pair_spread_sd, True, False)

    else: continue #move on to next iteration if there are no stationary series

    #profit calculation  

    open_trade_df = pair_df[pair_df['deviation_2sd'] == True].loc[30:] #so that we only consider data after the first 30 periods
    if len(open_trade_df.index) == 0:
        continue #move on to next iteration if there are no trading opportunities
        
    num_trades += 1
    
    open_trade_idx = open_trade_df.index[0]
    close_trade_idx = open_trade_idx + k
    
    if close_trade_idx > 59: 
        continue #move on to next iteration if trade cant be simulated

    #to work out what the direction of the trade should be
    #the spreads are always defined as horse_a - horse_b, so if the spread > average, btl horse_a and ltb horse_b. if below average, vice versa.
    
    if (pair_df['spread'].iloc[open_trade_idx] > pair_spread_mean and pair_spread_mean > 0) or (pair_df['spread'].iloc[open_trade_idx] < pair_spread_mean and pair_spread_mean < 0):
        #back to lay A
        bp_a = pair_df[horse_a + "_bp"].iloc[open_trade_idx]
        lp_a = pair_df[horse_a + "_lp"].iloc[close_trade_idx]

        win_side_a, loss_side_a = payout(bp_a, 1, lp_a, '?')

        #lay to back X
        lp_b = pair_df[horse_b + "_lp"].iloc[open_trade_idx]
        bp_b = pair_df[horse_b + "_bp"].iloc[close_trade_idx]

        win_side_b, loss_side_b = payout(bp_b, '?', lp_b, 1)

        profit += win_side_a + win_side_b
        
    else: 
        #lay to back Y
        lp_a = pair_df[horse_a + "_lp"].iloc[open_trade_idx]
        bp_a = pair_df[horse_a + "_bp"].iloc[close_trade_idx]

        win_side_a, loss_side_a = payout(bp_a, '?', lp_a, 1)
        
        #back to lay X
        bp_b = pair_df[horse_b + "_bp"].iloc[open_trade_idx]
        lp_b = pair_df[horse_b + "_lp"].iloc[close_trade_idx]

        win_side_b, loss_side_b = payout(bp_b, 1, lp_b, '?')

        profit += win_side_a + win_side_b
        
print(f"Profit over {n} iterations = {profit}. {num_pairs} pairs found and {num_trades} pairs trades made.")

C:\Users\Asus\anaconda\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


100 of 1000 iterations completed.
200 of 1000 iterations completed.
300 of 1000 iterations completed.
400 of 1000 iterations completed.
500 of 1000 iterations completed.
600 of 1000 iterations completed.
700 of 1000 iterations completed.
800 of 1000 iterations completed.
900 of 1000 iterations completed.
1000 of 1000 iterations completed.
Profit over 1000 iterations = -32.48763332543176. 408 pairs found and 310 pairs trades made.
